## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

## [Rubric](https://review.udacity.com/#!/rubrics/571/view) Points

### Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  

### Camera Calibration

#### 1. Briefly state how you computed the camera matrix and distortion coefficients. Provide an example of a distortion corrected calibration image.

[//]: # (Image References)

[image1]: ./output_images/undistort_output.png "Undistorted"

The code for this project is contained in the IPython notebook located in "./CarND-Advanced-Lane-Lines/CarND-Advanced-Lane-Lines-P4.ipynb"

I start by preparing "object points", which will be the (x, y, z) coordinates of the chessboard corners in the world. Here I am assuming the chessboard is fixed on the (x, y) plane at z=0, such that the object points are the same for each calibration image.  Thus, `objp` is just a replicated array of coordinates, and `objpoints` will be appended with a copy of it every time I successfully detect all chessboard corners in a test image.  `imgpoints` will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection.  

I then used the output `objpoints` and `imgpoints` to compute the camera calibration and distortion coefficients using the `cv2.calibrateCamera()` function.  I applied this distortion correction to the test image using the `cv2.undistort()` function and obtained this result: 

![Undistorted][image1]

The camera calibration matrix and distortion coefficients are stored in "wide_dist_pickle.p".

[image1]: ./output_images/undistort_test_image.png "Undistorted"
[image2]: ./output_images/color_transform.png "Color Transform"
[image3]: ./output_images/Sobel_X.png "Grdient Thrshold"
[image4]: ./output_images/color_threshold.png "Color Thrshold"
[image5]: ./output_images/color_combination.png "Color Combination"
[image6]: ./output_images/binary_combination.png "Binary Combination"
[image7]: ./output_images/warped_results.png "Warped Result"
[image8]: ./output_images/curved_results.png "Curved Result"
[image9]: ./output_images/improved_curved_results.png "Improved Curved Result"

### Pipeline (single image)

#### 1. Provide an example of a distortion-corrected image.

I used the results calculated by last step and apply `cv2.undistort()` to the test image. The result turns out:

![Undistorted][image1]

#### 2. Describe how (and identify where in your code) you used color transforms, gradients or other methods to create a thresholded binary image.  Provide an example of a binary image result.

Firstly, I use the `cv2.cvtColor()` function to convert the test image format from rgb to hls. In the lecture HLS and Threshold lecture, I have tried several color channels to extract the feature of the lane under different conditions. The S channel is still doing a fairly robust job of picking up the lines under very different color and contrast conditions, while the other selections look messy. Here's an example of my output for this step：

![Color Transform][image2]

Secondly, I apply the sobel gradient threshold to the S_channel binary image. The sobel x gradient threshold is between 20 and 100.

![Sobel X][image3]

Then I aplly the color thrshold to the S_channel binary image. The color threshold is between 170 and 255

![Color Threshold][image4]

The outputs are shown below. The final images color_combination and binary_combination are combinations of binary thresholding the S channel (HLS) and binary thresholding the result of applying the Sobel operator in the x direction on the original image. 

![Color Combination][image5]
![Binary Combination][image6]


#### 3. Describe how (and identify where in your code) you performed a perspective transform and provide an example of a transformed image.

To perform a perspective transform, I need to manually define the four source coordinates and set the target coordinates.

    # set the source coordinates
    src = np.float32([[200, 700],
                      [515, 500],
                      [780, 500],
                      [1100, 700]])

    # set the target coordinates
    dst = np.float32([[370,700],
                      [370,0],
                      [960,0],
                      [960,700]])
                      
Then I use `cv2.getPerspectiveTransform(src, dst)` to compute the transform matrix. After obtaining transform matrix,  `cv2.warpPerspective()` function is used to get the transformation result. Here's an example of my output for this step：

![Warped Result][image7]

The above resutls show that this pipeline is good at detecting straight line but except straight lines, there are many curves in the real road. The results below shows the pipeline used to perform on the curved line.

![Wapred Result][image8]

We can see that this pipeline performs not well on detecting the curved lines. In order to get better performance, I edit the source coordinates and target coordinates. 
    
    # set the new source coordinates
    src = np.float32([(575,464),
                      (707,464), 
                      (258,682), 
                      (1049,682)])
    
    # set the new target coordinates
    dst = np.float32([(450,0),
                     (w-450,0),
                     (450,h),
                     (w-450,h)])

![Wapred Result][image9]




## Image Preprocessing Pipeline 

The final image preprocessing pipeline is defined after repeated attempts.

def pipeline(img):
    # Undistort
    img_undistort = undistort(img)
    
    # Perspective Transform
    img_unwarp, M, Minv = unwarp(img_undistort, src, dst)

    # HLS L-channel Threshold (using default parameters)
    img_LThresh = hls_lthresh(img_unwarp)

    # Lab B-channel Threshold (using default parameters)
    img_BThresh = lab_bthresh(img_unwarp)
    
    # Combine HLS and Lab B channel thresholds
    combined = np.zeros_like(img_BThresh)
    combined[(img_LThresh == 1) | (img_BThresh == 1)] = 1
    return combined, Minv

[image1]: ./output_images/half_binary.png "Half"
[image2]: ./output_images/histogram.png "Histogram"
[image3]: ./output_images/sliding_windows.png "Sliding Windows"


## Implement Sliding Windows and Fit a Polynomial

#### 1. Describe how (and identify where in your code) you identified lane-line pixels and fit their positions with a polynomial?

1. Find the starting position of the left and right lane lines 

        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
        
        # Create an output image to draw on and visualize the result
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]//2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
        
![Half][image1]
![Histogram][image2]

2. Choose the number of sliding windows, set the width of the window and set minimum number of pixels found to recenter window.

        nwindows = 10
        margin = 60
        minpix = 40

3. Step through the windows one by one and identify the nonzero pixels in x and y within the window, If you found > minpix pixels, recenter next window.
        
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        
        
4. Extract left and right line pixel positions:     
        
        leftx = nonzerox[left_lane_inds]
        
        lefty = nonzeroy[left_lane_inds] 
        
        rightx = nonzerox[right_lane_inds]
        
        righty = nonzeroy[right_lane_inds]
5. Use `np.ployfit()` to fit a second order polynomial by the leftx, lefty and rightx, righty 
6. Display the fit result.
![sliding window][image3]


[image1]: ./output_images/skip_results.png "Skip Results"

## Skip the sliding windows step once you've found the lines
1. Choose the width of the margin around the previous polynomial to search


2. The different from sliding windows step.(Using the previous fit)

            left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
            right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
                    
3. Extract left and right line pixel positions:     
        
        leftx = nonzerox[left_lane_inds]
        
        lefty = nonzeroy[left_lane_inds] 
        
        rightx = nonzerox[right_lane_inds]
        
        righty = nonzeroy[right_lane_inds]
4. Use `np.ployfit()` to fit a second order polynomial by the leftx, lefty and rightx, righty 
5. Display the fit result.

![skip results][image1]

## Calculated the radius of curvature of the lane and the position of the vehicle with respect to center.

#### 1. Describe how (and identify where in your code) you calculated the radius of curvature of the lane and the position of the vehicle with respect to center.

The radius of curvature is calculated:

        left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
        
The vehicle with respect to center is calcualted by:

        car_position = bin_img.shape[1]/2
        l_fit_x_int = l_fit[0]*h**2 + l_fit[1]*h + l_fit[2]
        r_fit_x_int = r_fit[0]*h**2 + r_fit[1]*h + r_fit[2]
        lane_center_position = (r_fit_x_int + l_fit_x_int) /2
        center_dist = (car_position - lane_center_position) * xm_per_pix
        
        
The results of one sample image are: 
        
        left curvature: 394.56879928 m 
        right curvature: 459.859255137 m
        offset : 2.3981481481481484(left) 

[image1]: ./output_images/draw_results.png "Draw Results"

## Draw the detected results on the original image

After finding the lane line and calculating the curvature and offset. I defined `draw_lane()` funtion and `draw_data` to draw the detected results on the original image.

![Draw Results][image1]

## Completed Advanced Lane Find Pipeline

1. Define a class to receive the characteristics of each line detection

The line class has two functions: one is the init function and the other is the add_fit function:

    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        
        #polynomial coefficients for the most recent fit
        self.current_fit = []  
        
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        
        #number of detected pixels
        self.px_count = None  
    
    def add_fit(self, fit, inds):
        # add a found fit to the line, up to n
        if fit is not None:
            if self.best_fit is not None:
                # if we have a best fit, see how this new fit compares
                self.diffs = abs(fit-self.best_fit)
            if (self.diffs[0] > 0.001 or \
                self.diffs[1] > 1.0 or \
                self.diffs[2] > 100.0 and \
                len(self.current_fit) > 0):
                
                self.detected = False
            else:
                self.detected = True
                self.px_count = np.count_nonzero(inds)
                self.current_fit.append(fit)
                if len(self.current_fit) > 5:
                    # throw out old fits, keep newest n
                    self.current_fit = self.current_fit[len(self.current_fit)-5:]
                self.best_fit = np.average(self.current_fit, axis=0)
        # or remove one from the history, if not found
        else:
            self.detected = False
            if len(self.current_fit) > 0:
                # throw out oldest fit
                self.current_fit = self.current_fit[:len(self.current_fit)-1]
            if len(self.current_fit) > 0:
                # if there are still any fits in the queue, best_fit is their average
                self.best_fit = np.average(self.current_fit, axis=0)


Finally, Use the Line Class and Lane Find Pipeline function to process the video:

    l_line = Line()
    r_line = Line()
    video_output1 = 'project_video_output.mp4'
    video_input1 = VideoFileClip('project_video.mp4')
    processed_video = video_input1.fl_image(process_img)
    %time processed_video.write_videofile(video_output1, audio=False)

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

Risk:
1. The results of perspective transform will lead the pipeline fail to detect the right curved lane.
2. The window margin width and mini_pixel numbers will lead the pipeline fail to detec the lane. If the width is too big, the window will including pixels which are not part of the lane and will get the wrong ploynomial fit.
3. The choice of gradient and color threshold is also a key factor that will influence the detected results.

Improvement:
1. Dynamically selecting threshold parameters based on the resulting number of activated pixels